In [1]:
import utils
from urllib.parse import urlparse

import os, shutil
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.mysql import LONGTEXT

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
hostname="localhost"
dbname="apptio"
uname="sreejith"
pwd="password"
table = "events"


engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [3]:
connection = engine.raw_connection()
cursor = connection.cursor()

output_table = "events_clean"
command = "DROP TABLE IF EXISTS {};".format(output_table)
cursor.execute(command)

0

### Understanding the Clicked events
---

In [4]:
# url, documentReferer are the useful fields 

In [5]:
def get_domain(long_url):
    
    referrer_details = urlparse(str(long_url))
    return referrer_details.netloc

In [12]:
sql = "select documentReferer, url from " + table 
for chunk in pd.read_sql(sql, engine, chunksize=10000):
    
    chunk["from"] = chunk["documentReferer"].apply(get_domain)
    chunk["to"] = chunk["url"].apply(get_domain)
    
    del chunk["documentReferer"]
    del chunk["url"]
        
    events_clean_type_map = {x:types.TEXT for x in chunk.columns.tolist()}
    chunk.to_sql(name=output_table, con=engine, dtype=events_clean_type_map, index=False, if_exists='append')
    
    chunk = None
    del chunk

KeyboardInterrupt: 

In [ ]:
# http://www.imdb.com/title/tt0364845/episodes?ref_=tt_ov_epl
# http://www.imdb.com/title/tt0364845/?ref_=nv_sr_1


# 'http://s.amazon-adsystem.com/iu3?d=imdb.com&amp;a1=0101aa0ea84c4cb084de5eb8684bafc4e260f19884e8b432dbcab2fbfb554b3d26e9&amp;a2=0101f074615d8fd335a2e171a738c8802f2264442fafb19f9d89c4556aae76ddd76c&amp;pId=&amp;r=1&amp;rP=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2Feprate%3Fref_%3Dttep_ql_3&amp;encoding=server&amp;cb=667171775044'
# 'http://www.imdb.com/title/tt0364845/eprate?ref_=ttep_ql_3'
# 'https://www.facebook.com/widgets/like.php?width=280&amp;show_faces=1&amp;layout=standard&amp;href=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2F&amp;colorscheme=light'

In [10]:
get_referer_url("http://www.imdb.com/title/tt0364845/episodes?ref_=tt_ov_epl")

ParseResult(scheme='http', netloc='www.imdb.com', path='/title/tt0364845/episodes', params='', query='ref_=tt_ov_epl', fragment='')


In [11]:
get_referer_url("https://www.facebook.com/widgets/like.php?width=280&amp;show_faces=1&amp;layout=standard&amp;href=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2F&amp;colorscheme=light")

ParseResult(scheme='https', netloc='www.facebook.com', path='/widgets/like.php', params='', query='width=280&amp;show_faces=1&amp;layout=standard&amp;href=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2F&amp;colorscheme=light', fragment='')


In [12]:
get_referer_url("http://s.amazon-adsystem.com/iu3?d=imdb.com&amp;a1=0101aa0ea84c4cb084de5eb8684bafc4e260f19884e8b432dbcab2fbfb554b3d26e9&amp;a2=0101f074615d8fd335a2e171a738c8802f2264442fafb19f9d89c4556aae76ddd76c&amp;pId=&amp;r=1&amp;rP=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2Feprate%3Fref_%3Dttep_ql_3&amp;encoding=server&amp;cb=667171775044")

ParseResult(scheme='http', netloc='s.amazon-adsystem.com', path='/iu3', params='', query='d=imdb.com&amp;a1=0101aa0ea84c4cb084de5eb8684bafc4e260f19884e8b432dbcab2fbfb554b3d26e9&amp;a2=0101f074615d8fd335a2e171a738c8802f2264442fafb19f9d89c4556aae76ddd76c&amp;pId=&amp;r=1&amp;rP=http%3A%2F%2Fwww.imdb.com%2Ftitle%2Ftt0364845%2Feprate%3Fref_%3Dttep_ql_3&amp;encoding=server&amp;cb=667171775044', fragment='')
